In [42]:

! uv pip install langchain langchain-google-genai tiktoken rapidocr-onnxruntime python-dotenv langchain-community

Using Python 3.12.5 environment at: C:\Users\manas\OneDrive\Desktop\multi-doc-chat\.venv
Audited 6 packages in 2.01s


In [43]:
! uv pip install -q sentence_transformers

In [44]:
! uv pip install -qU langchain-huggingface

In [45]:

import os
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()
os.environ["GOOGLE_API_KEY"] =os.getenv("GOOGLE_API_KEY")


llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",

)

In [46]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("rag_data.txt", encoding="utf-8")
docs = loader.load()



In [47]:
data=docs[0].page_content

In [56]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,       # max characters per chunk
    chunk_overlap=20,     # how much text to overlap between chunks
    separators=["\n\n", "\n", " ", ""],  # default fallback split hierarchy
)


chunks = splitter.split_documents(docs)

In [57]:
chunks

[Document(metadata={'source': 'rag_data.txt'}, page_content='Recent AI developments — summary (compiled Nov 6, 2025)'),
 Document(metadata={'source': 'rag_data.txt'}, page_content='Overview:'),
 Document(metadata={'source': 'rag_data.txt'}, page_content='- This file summarizes recent developments (2024–2025) in large language models (LLMs), embedding models, and agentic AI (agent frameworks, products, and industry trends). Each major claim includes a'),
 Document(metadata={'source': 'rag_data.txt'}, page_content='claim includes a short citation tag for reference to original reporting.'),
 Document(metadata={'source': 'rag_data.txt'}, page_content='1) New / notable LLM releases and trends'),
 Document(metadata={'source': 'rag_data.txt'}, page_content="- LLaMA / Llama 4 family: Meta's Llama 4 (and follow-ups like Llama 4 Scout / Maverick) expanded multimodal capabilities and used mixture-of-experts architectures to scale efficiency and add"),
 Document(metadata={'source': 'rag_data.txt'}

In [41]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [52]:
! uv pip install faiss-cpu

Using Python 3.12.5 environment at: C:\Users\manas\OneDrive\Desktop\multi-doc-chat\.venv
Resolved 3 packages in 767ms
Prepared 1 package in 13.30s
Installed 1 package in 91ms
 + faiss-cpu==1.12.0


In [ ]:
from langchain_community.vectorstores import FAISS

In [58]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

# vector_store = FAISS(
#     embedding_function=embeddings,
#     index=index,
#     docstore=InMemoryDocstore(),
#     index_to_docstore_id={},
# )
vectorstore=FAISS.from_documents(chunks, embeddings)

In [59]:
retriever=vectorstore.as_retriever()

In [60]:
doc=vectorstore.similarity_search_with_relevance_scores(query="what is the recent embedding model",k=5)
doc

[(Document(id='fb762ded-dbfd-4fd1-9c36-caac4a6f6a32', metadata={'source': 'rag_data.txt'}, page_content='2) Embedding models (2024–2025 highlights)'),
  np.float32(0.79484797)),
 (Document(id='b0444df2-2a95-41d4-bb7a-3f3686b69e1c', metadata={'source': 'rag_data.txt'}, page_content="- Specialized and on-device embedding releases (e.g., Google's EmbeddingGemma announced Sept 2025) targeted small-footprint, multilingual embeddings for on-device semantic search and RAG scenarios."),
  np.float32(0.77727056)),
 (Document(id='6c8b2403-1488-47b5-88ff-fd6bd1883663', metadata={'source': 'rag_data.txt'}, page_content='- OpenAI: New embedding models and API updates (text-embedding-3 announcement). \ue200cite\ue202turn0search5\ue201'),
  np.float32(0.76711977)),
 (Document(id='48e28468-be6e-4de3-85b5-48a921626e1e', metadata={'source': 'rag_data.txt'}, page_content='- This file summarizes recent developments (2024–2025) in large language models (LLMs), embedding models, and agentic AI (agent framew

In [62]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough



template="""you are an ai assistant .your job is to answer the query of user. you are given context .based on the context ans the queries. use 50 words to answer the query.
here is the context:{context}
here is the query:{query}
"""
prompt=ChatPromptTemplate.from_template(template)

chain=(
    {"context":retriever,"query":RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)



In [63]:

chain.invoke("tell me about recent embedding model developed")

"Recent embedding model developments (2024–2025) include OpenAI's new embedding models and API updates, such as `text-embedding-3`. Google also announced `EmbeddingGemma` (Sept 2025), which focuses on specialized, small-footprint, multilingual embeddings for on-device semantic search and RAG scenarios. These advancements aim to enhance performance and utility in various AI applications."